In [1]:
import time
import copy


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset

In [2]:
transformer_train = T.Compose([
    T.RandomCrop([178, 178]),
    T.RandomHorizontalFlip(),
    T.Resize([64, 64]),
    T.ToTensor(),
])

transformer = T.Compose([
    T.CenterCrop([178, 178]),
    T.Resize([64, 64]),
    T.ToTensor(),
])

get_smile = lambda attr: attr[18].to(torch.float32)

In [3]:
train_data = torchvision.datasets.CelebA('./', split='train', target_transform=get_smile, transform=transformer_train)
valid_data = torchvision.datasets.CelebA('./', split='valid', target_transform=get_smile, transform=transformer)
test_data = torchvision.datasets.CelebA('./', split='test', target_transform=get_smile, transform=transformer)

In [4]:
# lowering number of samples
train_data = Subset(train_data, torch.arange(16_000))
valid_data = Subset(valid_data, torch.arange(1_000))
test_data = Subset(test_data, torch.arange(1_000))

In [5]:
batch_size = 256

train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=8)
valid_dl = DataLoader(valid_data, batch_size, shuffle=False, num_workers=8)
test_dl = DataLoader(test_data, batch_size, shuffle=False, num_workers=8)

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        conv_1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.MaxPool2d(2)
        )
        conv_2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d(2)
        )
        conv_3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.MaxPool2d(2)
        )
        conv_4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
        )
        convs = nn.Sequential(*[conv_1, conv_2, conv_3, conv_4])
        
        avg_flatten = nn.Sequential(
            nn.AvgPool2d(kernel_size=8),
            nn.Flatten()
        )
        
        fc_1 = nn.Sequential(
            nn.Linear(256, 256),
            nn.LeakyReLU()
        )
        fc_2 = nn.Sequential(
            nn.Linear(256, 64),
            nn.LeakyReLU()
        )
        fc_3 = nn.Linear(64, 1)
        fcs = nn.Sequential(*[fc_1, fc_2, fc_3])
        
        sigmoid = nn.Sigmoid()
        
        self.model = nn.Sequential(*[convs, avg_flatten, fcs, sigmoid])
        
    def forward(self, x):
        return self.model(x)

In [10]:
def train_model(model, criterion, optimizer, data_loader, num_epochs, epoch_print=1):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs + 1):
        if epoch == 1 or epoch % epoch_print == 0:
            print(f'\nEpoch {epoch}/{num_epochs}')
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data_loader[phase]:

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)[:, 0]
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum((outputs > 0.5) == labels.data)

            epoch_loss = running_loss / len(data_loader[phase].dataset)
            epoch_acc = running_corrects.double() / len(data_loader[phase].dataset)

            if epoch == 1 or epoch % epoch_print == 0:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

                
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [11]:
model = CNN()
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters())

In [12]:
model = train_model(model, criterion, optimizer, {'train': train_dl, 'val': valid_dl}, 20)


Epoch 1/20
----------
train Loss: 0.5561 Acc: 0.7061
val Loss: 0.7070 Acc: 0.6430

Epoch 2/20
----------
train Loss: 0.4197 Acc: 0.8043
val Loss: 0.4346 Acc: 0.8030

Epoch 3/20
----------
train Loss: 0.3316 Acc: 0.8531
val Loss: 0.3625 Acc: 0.8430

Epoch 4/20
----------
train Loss: 0.2848 Acc: 0.8723
val Loss: 0.7092 Acc: 0.6740

Epoch 5/20
----------
train Loss: 0.2666 Acc: 0.8839
val Loss: 1.0630 Acc: 0.6270

Epoch 6/20
----------
train Loss: 0.2475 Acc: 0.8922
val Loss: 0.3316 Acc: 0.8530

Epoch 7/20
----------
train Loss: 0.2418 Acc: 0.8934
val Loss: 0.6636 Acc: 0.7450

Epoch 8/20
----------
train Loss: 0.2314 Acc: 0.8989
val Loss: 0.3352 Acc: 0.8570

Epoch 9/20
----------
train Loss: 0.2337 Acc: 0.8962
val Loss: 0.8579 Acc: 0.6780

Epoch 10/20
----------
train Loss: 0.2249 Acc: 0.9019
val Loss: 0.2894 Acc: 0.8710

Epoch 11/20
----------
train Loss: 0.2226 Acc: 0.9029
val Loss: 0.2658 Acc: 0.8900

Epoch 12/20
----------
train Loss: 0.2175 Acc: 0.9059
val Loss: 0.5089 Acc: 0.8130



In [13]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, labels in loader:model = train_model(model, criterion, optimizer, {'train': train_dl, 'val': valid_dl}, 20)
            
            outputs = model(x)[:, 0]
            num_correct += torch.sum((outputs > 0.5) == labels.data)
            num_samples += outputs.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

In [14]:
check_accuracy(test_dl, model)

Got 895 / 1000 with accuracy 89.50


In [15]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

471681